# Exercício Pipeline

- Utilize o dataset do Gapminder `gm_2008_region.csv`
- Prepare um pipeline com os seguintes passos:
    - `'imputation'`, que usa Imputer() com a estratégea `'mean'` para adicionar dados faltantes `('NaN')` usando a média da coluna.
    - `'scaler'`, que normaliza as features usando StandardScaler().
    - `'elasticnet'`, que instancia um regressor ElasticNet().
- Especifique o espaço do hyperparametro para o l1 ratio usando a seguinte notação: `'step_name__parameter_name'`. Onde, step_name é `elasticnet`, e the `parameter_name` é `l1_ratio`. 
`parameters = {'elasticnet__l1_ratio':np.linspace(0,1,30)}`
- Crie os training e test sets, com 40% dos dados usados para teste. Use random state 42.
- Instancie GridSearchCV com o pipeline e o espaço de hyperparametro, Use 3-fold cross-validation (esse é o padrão)
- Fit o GridSearchCV object para o training set.
- Compute o score padrão e os melhores parametros e imprimna. 

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

In [4]:
df = pd.read_csv('gm_2008_region.csv')
df.head()

,population,fertility,HIV,CO2,BMI_male,GDP,BMI_female,life,child_mortality,Region
0,34811059.0,2.73,0.1,3.328945,24.59620,12314.0,129.9049,75.3,29.5,Middle East & North Africa
1,19842251.0,6.43,2.0,1.474353,22.25083,7103.0,130.1247,58.3,192.0,Sub-Saharan Africa
2,40381860.0,2.24,0.5,4.785170,27.50170,14646.0,118.8915,75.5,15.4,America
3,2975029.0,1.40,0.1,1.804106,25.35542,7383.0,132.8108,72.5,20.0,Europe & Central Asia
4,21370348.0,1.96,0.1,18.016313,27.56373,41312.0,117.3755,81.5,5.2,East Asia & Pacific


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   population       139 non-null    float64
 1   fertility        139 non-null    float64
 2   HIV              139 non-null    float64
 3   CO2              139 non-null    float64
 4   BMI_male         139 non-null    float64
 5   GDP              139 non-null    float64
 6   BMI_female       139 non-null    float64
 7   life             139 non-null    float64
 8   child_mortality  139 non-null    float64
 9   Region           139 non-null    object 
dtypes: float64(9), object(1)
memory usage: 11.0+ KB


In [17]:
X = df.drop(['life', 'Region'], axis=1).values
y = df['life'].values

In [18]:
steps = [('imputation', SimpleImputer(missing_values=np.nan, strategy='mean')), ('scaler', StandardScaler()), ('elasticnet', ElasticNet())]
pipeline = Pipeline(steps)
parameters = {'elasticnet__l1_ratio':np.linspace(0,1,30)}

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [23]:
cv = GridSearchCV(pipeline, param_grid=parameters, cv=3)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

/home/natannobre/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.344e+02, tolerance: 4.519e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/home/natannobre/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.626e+02, tolerance: 5.124e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemen

In [24]:
print(cv.best_params_)
print(cv.score(X_test, y_test))

{'elasticnet__l1_ratio': 1.0}
0.8862016570888216
